In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import models
from keras import applications
from keras import optimizers
import matplotlib.pyplot as plt
from google.colab import drive

Using TensorFlow backend.


In [2]:
drive.mount('/content/drive')

parent_dir = '/content/drive/My Drive/Colab Notebooks'
tr_img_dir = 'training'
ts_img_dir = 'validation'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_dir = os.path.join(parent_dir, tr_img_dir)
validation_dir = os.path.join(parent_dir, ts_img_dir)

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary')

Found 0 images belonging to 2 classes.


In [6]:
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary')

Found 3901 images belonging to 2 classes.


In [7]:
conv_base = applications.resnet50.ResNet50(include_top=False,
                                           weights='imagenet',
                                           input_tensor=None,
                                           input_shape=(224, 224, 3),
                                           pooling=None, classes=2)

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step


In [8]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
opt = optimizers.Adam(lr=0.00025, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()
history = model.fit_generator(train_generator,
                              steps_per_epoch=80,
                              epochs=30,
                              validation_data=(validation_generator),
                              validation_steps=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               12845184  
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 36,433,025
Trainable params: 36,379,905
Non-trainable params: 53,120
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/30


ZeroDivisionError: ignored

In [0]:
model.save('/content/drive/My Drive/Colab Notebooks/ResNet50Bin2large.h5')

In [0]:
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

In [0]:
epochs = range(1, len(loss) + 1)

In [0]:
plt.figure(1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [0]:
plt.figure(2)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [0]:
plt.show()

In [0]:
import numpy as np
Y_pred = model.predict_generator(validation_generator, 50+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['Speech', 'Babble', 'Environment']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))